<a href="https://colab.research.google.com/github/crystaljwang/tm10007_group_3/blob/main/main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All necessary libraries
All libraries used in this code are run here

In [8]:
# ----- Import necessary libraries -----
from pathlib import Path
import pandas as pd
import math

from scipy.stats import shapiro 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA

# Imports

The code below loads the GIST data from GitHub. This was used once to create the test and train set. It is not needed to run this again

In [20]:
# # Run this to use from colab environment
# !pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

# # Run this to use from colab environment
# !git clone https://github.com/jveenland/tm10007_ml.git

# %cd /content/tm10007_ml/worcgist

ERROR: git+https://github.com/crystaljwang/tm10007_group_3.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [9]:
# # ---- Import data -----
# dir = Path('.') / 'GIST_radiomicFeatures.csv'
# data = pd.read_csv(dir, index_col=0)

# print(f'The number of samples: {len(data.index)}')
# print(f'The number of columns: {len(data.columns)}')

The number of samples: 246
The number of columns: 494


# Splitting the data
This code was used to split the data the first time. To make sure we were all using the same train and test data we wrote csv files for the train and test data, that we are loading again below. This code is used once, and is not necessary to run again.

In [16]:
# # Replace label values from string to binary
# data['label'] = data['label'].replace({'GIST': 1, 'non-GIST': 0})

# # Separate the features and labels
# X = data.drop(['label'], axis=1)
# y = data['label']

# # Split the data into random train and test sets
# X_train_tot, X_test, y_train_tot, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Write the X_test DataFrame to a CSV file
# X_train_tot.to_csv('X_train_tot.csv', index=False)
# X_test.to_csv('X_test.csv', index=False)
# y_train_tot.to_csv('y_train_tot.csv', index=False)
# y_test.to_csv('y_test.csv', index=False)

fatal: destination path 'tm10007_group_3' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/tm10007_group_3'
/content/tm10007_ml/worcgist
/content/tm10007_ml/worcgist


# Load the train data

In [28]:
#load split data from our repository
url = "https://raw.githubusercontent.com/crystaljwang/tm10007_group_3/main/data/"
X_train_tot = pd.read_csv(url + 'X_train_tot.csv')
y_train_tot = pd.read_csv(url + 'y_train_tot.csv')
X_test = pd.read_csv(url + 'X_test.csv')
y_test = pd.read_csv(url + 'y_test.csv')

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_tot, y_train_tot, test_size=0.15, random_state=42)

# Exploring the data

In [29]:
# Percentage GIST vs Non-GIST
counts = y_train.value_counts(normalize=True)
percentage_nongist = counts[0] * 100
percentage_gist = counts[1] * 100

print(f'Percentage of non-GIST in training set: {percentage_nongist:.2f}%')
print(f'Percentage of GIST in training set: {percentage_gist:.2f}%')

# Check for missing data
if X_train.isnull().sum().sum() > 0:
    print('Missing data found.')
    exit()
else:
    print('No missing data found.')

# Check for categorial values
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
if len(categorical_cols) > 0:
    print(f'Categorical columns found: {categorical_cols}')
    exit()
else:
    print('No categorical columns found.')

# Perform Shapiro-Wilk test for normality
p_values_above_threshold = []
for col in X_train.columns:
  stat, p = shapiro((X_train))
  p_values_above_threshold.append(p > 0.05)
  #print(f'Shapiro test for column {col}: statistic = {stat:.3f}, p-value = {p:.3f}')

percent_above_threshold = sum(p_values_above_threshold) / len(p_values_above_threshold) * 100
print(f'{percent_above_threshold:.1f} percent of the data is normally distributed.')

#print('stat=%.3f, p=%.3f\n' % (stat, p))

Percentage of non-GIST in training set: 51.20%
Percentage of GIST in training set: 48.80%
No missing data found.
No categorical columns found.


/usr/local/lib/python3.9/dist-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


0.0 percent of the data is normally distributed.


# Preprocessing

In [31]:
# ----- Outliers -----

def replace_outliers(data):
    """
    Replaces the outliers in a DataFrame with the lower or upper bound.

    :param data: The DataFrame to be filtered
    :return: A new DataFrame with the outliers replaced by the lower or upper bound for each column
    """
    # Calculate the lower and upper bounds based on each column's median and interquartile range
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace the outliers with the lower or upper bound
    for col in data.columns:
        data[col] = data[col].apply(lambda x: upper_bound[col] if x > upper_bound[col] else x)
        data[col] = data[col].apply(lambda x: lower_bound[col] if x < lower_bound[col] else x)

    return data

# Replace the outliers in each column with the lower or upper bound
X_train = replace_outliers(X_train)

In [32]:
# Data scaling
scaler = MinMaxScaler()  # define scaler

X_train = scaler.fit_transform(X_train)  # fit scaler on train set
X_val = scaler.transform(X_val)  # apply fitted scaler on validation set
X_test = scaler.transform(X_test)  # apply fitted scaler on test set

In [33]:
# Remove all constant (zero-variance) features
X_train = pd.DataFrame(X_train)
zero_var_filter = VarianceThreshold(threshold=0)

# Fit on train data
zero_var_filter.fit(X_train)
zero_var_columns = [column for column in X_train.columns if column not in X_train.columns[zero_var_filter.get_support()]]

# Apply on train, validation and test data
X_train = zero_var_filter.transform(X_train)
X_val = zero_var_filter.transform(X_val)
X_test = zero_var_filter.transform(X_test)

In [34]:
# ----- Feature selection -----
lasso_selector = SelectFromModel(estimator=Lasso(alpha=10**(-10), max_iter=1000), threshold='median')
lasso_selector.fit(X_train, y_train)
lasso_list = [column for column in pd.DataFrame(X_train).columns[lasso_selector.get_support()]]
n_original = X_train.shape[1]

X_train = lasso_selector.transform(X_train)
n_selected = X_train.shape[1]
print(f"Selected {n_selected} from {n_original} features.")

Selected 228 from 455 features.


In [36]:
# ----- Feature extraction -----
from sklearn.decomposition import PCA

# 95% variance
pca = PCA(n_components = 0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)

print(f"Selected {X_train.shape[1]} features to be used for classification.")

Selected 50 features to be used for classification.
